<a href="https://colab.research.google.com/github/Mageed-Ghaleb/OptimizationSystems-Course/blob/main/Lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3 — Convex Optimization with CVXPY

**Topics:** convex modeling mindset, disciplined convex programming (DCP), portfolio optimization with constraints, and a robustified norm-ball example (SOCP).

**How to use this notebook**
- Read the markdown, run the code cells in order.
- When you see **TODO**, complete that part.

**Deliverables**
- A completed notebook (export as HTML or PDF) with answers to all questions.
- A short reflection (at the end) interpreting results and identifying binding constraints.


## 0. Environment setup
This lab uses CVXPY. If you are running locally, you may need to install it first.


In [ ]:
# If needed, install dependencies (run once)
# !pip install -U cvxpy numpy pandas matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

np.set_printoptions(precision=4, suppress=True)


## 1. Quick warm-up: what makes a problem 'convex' in practice?
In this course, we care about convexity because convex problems are reliably solvable at scale and have strong optimality guarantees.

### 1.1 DCP sanity check in CVXPY
CVXPY enforces a set of modeling rules called disciplined convex programming (DCP). In this lab we will:
- build problems that are DCP-compliant
- learn how to detect non-DCP expressions


In [ ]:
x = cp.Variable()
y = cp.Variable()

# Example A: a convex objective with an affine constraint (should be DCP)
prob_ok = cp.Problem(
    cp.Minimize(cp.square(x - y)),
    [x + y >= 0]
)

print("Problem is DCP:", prob_ok.is_dcp())


### TODO 1
Create a small problem that is **not** DCP-compliant (for example: minimize a concave function, or maximize a convex function).
Then call `.is_dcp()` and confirm it returns False.


In [ ]:
# TODO 1: create a non-DCP problem and check is_dcp()

# Example idea (you can change it):
# prob_bad = cp.Problem(cp.Minimize(cp.sqrt(x)), [x >= 0])

# print("Problem is DCP:", prob_bad.is_dcp())


## 2. Portfolio optimization with constraints (convex QP)
We will solve a simple mean–variance style portfolio model with long-only constraints.

### 2.1 Data generation (synthetic)
To keep the lab self-contained, we generate synthetic expected returns and a positive semidefinite covariance matrix.


In [ ]:
rng = np.random.default_rng(7)

n_assets = 8
mu = rng.normal(loc=0.10, scale=0.05, size=n_assets)  # expected returns

# Create a random PSD covariance matrix
A = rng.normal(size=(n_assets, n_assets))
Sigma = (A.T @ A) / n_assets
Sigma = Sigma / np.max(np.diag(Sigma)) * 0.05  # scale variances

assets = [f"Asset {i+1}" for i in range(n_assets)]
pd.DataFrame({"asset": assets, "mu": mu}).style.format({"mu": "{:.3f}"})


### 2.2 Model
We choose portfolio weights w (length n_assets).

Common constraints:
- fully invested: sum(w) = 1
- long-only: w >= 0
- concentration limit: w <= w_max

Objective (one option): maximize expected return minus risk penalty
maximize   mu^T w  -  lambda * w^T Sigma w

This is a convex optimization problem in CVXPY (quadratic term uses `quad_form`).


In [ ]:
w = cp.Variable(n_assets, nonneg=True)
lam = cp.Parameter(nonneg=True, value=1.0)  # risk aversion
w_max = 0.35

objective = cp.Maximize(mu @ w - lam * cp.quad_form(w, Sigma))
constraints = [
    cp.sum(w) == 1,
    w <= w_max
]

prob = cp.Problem(objective, constraints)
print("DCP:", prob.is_dcp())


### 2.3 Solve for a single lambda
After solving, inspect:
- objective value
- weights
- which constraints are binding


In [ ]:
lam.value = 2.0
prob.solve()

print("status:", prob.status)
print("objective:", prob.value)

w_star = w.value
result = pd.DataFrame({"asset": assets, "weight": w_star, "mu": mu})
result = result.sort_values("weight", ascending=False)
result


### 2.4 Identify binding constraints
A constraint is numerically 'binding' if it is very close to equality at the solution.
Check:
- sum(w) == 1
- any weights close to 0 (long-only lower bound)
- any weights close to w_max (upper bound)


In [ ]:
tol = 1e-6

sum_w = w_star.sum()
near_zero = np.where(w_star <= 1e-4)[0]
near_max = np.where(w_star >= w_max - 1e-4)[0]

print("sum(w):", sum_w)
print("weights near 0:", [assets[i] for i in near_zero])
print("weights near w_max:", [assets[i] for i in near_max])


### TODO 2
Change w_max (for example 0.20 or 0.50) and re-solve.
Write 3 to 5 lines interpreting what changed and why.


In [ ]:
# TODO 2: modify w_max, re-solve, and interpret.


### 2.5 Efficient frontier style sweep (optional)
We sweep lambda values and plot expected return vs risk (standard deviation).


In [ ]:
lams = np.logspace(-2, 2, 20)
rets = []
risks = []

for lv in lams:
    lam.value = float(lv)
    prob.solve(warm_start=True)
    wv = w.value
    rets.append(mu @ wv)
    risks.append(np.sqrt(wv.T @ Sigma @ wv))

plt.figure()
plt.plot(risks, rets, marker="o")
plt.xlabel("Risk (std dev)")
plt.ylabel("Expected return")
plt.title("Tradeoff curve from lambda sweep (synthetic)")
plt.show()


## 3. Robustified norm-ball constraint example (SOCP)
Goal: see how an uncertainty set (a norm ball) leads to a deterministic convex constraint.

Setup:
We want to minimize c^T x subject to a_i^T x <= b_i for each i, but each a_i is uncertain:
a_i + u_i, where u_i has 2-norm <= 1.

In that case the robust constraint becomes:
a_i^T x + ||x||_2 <= b_i.

This is a second-order cone constraint.


In [ ]:
rng = np.random.default_rng(10)

n = 5   # dimension of decision variable
m = 6   # number of constraints

c = rng.normal(size=n)
A = rng.normal(size=(m, n))
b = rng.uniform(low=0.5, high=2.0, size=m)

x = cp.Variable(n)

constraints = []
for i in range(m):
    constraints.append(A[i, :] @ x + cp.norm(x, 2) <= b[i])

prob_rob = cp.Problem(cp.Minimize(c @ x), constraints)
print("DCP:", prob_rob.is_dcp())


### 3.1 Solve and interpret
Inspect:
- status and objective
- x
- which constraints are tight


In [ ]:
prob_rob.solve()

print("status:", prob_rob.status)
print("objective:", prob_rob.value)
x_star = x.value
print("x*:", x_star)

# check slack for each constraint
slacks = b - (A @ x_star + np.linalg.norm(x_star))
pd.DataFrame({"i": range(m), "slack": slacks}).sort_values("slack")


### TODO 3
Make the robust problem harder or easier by changing b.
For example:
- multiply b by 0.8 (harder)
- multiply b by 1.2 (easier)
Re-solve and note whether the problem becomes infeasible or how the solution changes.


In [ ]:
# TODO 3: change b (e.g., b = 0.8*b) and re-solve.


## 4. Submission checklist
Before you submit:
1. All TODOs completed.
2. You can explain why each model is convex and DCP-compliant.
3. You identified at least one binding constraint in each of the two main models.

### Reflection questions (answer in markdown)
1. In the portfolio model, which constraints were binding and what do they mean?
2. In the robust model, what role does the norm term play?
3. What is one modeling mistake that leads to a non-DCP problem?


## 5. References
- CVXPY documentation and tutorial: https://www.cvxpy.org/
- CVXPY DCP tutorial: https://www.cvxpy.org/tutorial/dcp/
- CVXPY SOCP robust LP example: https://www.cvxpy.org/examples/basic/socp.html
